## Airline AI Assistant

In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


## Using and external function (Tools)

In [5]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [7]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [8]:
# There's a particular dictionary structure that's required to describe the function:
# 


price_function = {
    "name": "get_ticket_price",
     # describe the funciton becaus ethes ewhat it wiill given to the llm 
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    # have one parameter which desctonation_city
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
# And this is included in a list of tools
tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use the Tool
Here's how the new chat function looks:

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)  # where as you can see I have added tools to it
    # by ading tool and descrption property it will add to llm the abilty thanks ot descptri to stop and run the fucniton defined
    
    
    if response.choices[0].finish_reason=="tool_calls":  #  this it will stop and say the one of tool provide it with output
        # what we want to do is ot collect the message from gpt 4.0
        message = response.choices[0].message
        response, city = handle_tool_call(message)  # we unpack the function and result of calling  
        messages.append(message)  
        messages.append(response)
        # send back to open ai
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [11]:
#  function handle_tool_call

def handle_tool_call(message):
    # Retrieve the first tool call from the message (assuming only one tool is used)
    tool_call = message.tool_calls[0]
    
    # Parse the arguments of the tool call (in string format) into a dictionary
    arguments = json.loads(tool_call.function.arguments)
    
    # Extract the destination city from the arguments
    destination_city = arguments.get('destination_city')
    
    # Fetch the ticket price for the provided city
    ticket_price = get_ticket_price(destination_city)
    
    # Prepare the response to be sent back to the tool
    response = {
        "role": "tool",  # Indicates this is a tool response
        "content": json.dumps({
            "destination_city": destination_city,  # Return the destination city
            "price": ticket_price  # Include the calculated ticket price
        }),
        "tool_call_id": tool_call.id  # Include the ID of the tool call
    }
    
    # Return the response and the destination city for further use if needed
    return response, destination_city


In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for London
Tool get_ticket_price called for Berlin
